In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

# Production App

This notebook will introduce you to some of the concepts necessary to build the Production App project as a team.  Note that this is a rather opinionated guide which promotes the way we will be pursuing this project, there are many other successful ways of engineering software and your production practices will vary wildly depending on factors like the size the team, nature of your project, and most importantly, the opinions of your superiors.

## Project Overview


The goal of this project is to work as a team to produce a web app which is centered around a general predictive model for time series applied to stock market data (obtained from Quandl).  The model has been built and deployed by the instructors and is based on the `fbprophet` package from Facebook.  We have wrapped some small functionality of the `fbprophet` package in a flask app to expose a REST service, containerized it, and deployed it on a Kubernetes cluster (Please ask about these things if you are interested!).  The idea here is that the model part has been built and exposed as a service. You will need to use it (flaws and all).  
This project will get you used to working on version controlled software which has a production deploy system on a good size team.  You will need to work with your peers to do the following:

1. Choose a direction for the app.  The general idea is to make stock predictions, many different ways you might choose to take this.  We have left this open ended, be creative and have some fun!
2. Once you have a direction, break the large direction into smaller tasks and work in teams on each task (Zach and Dylan will help here).
3. Start coding, testing, deploying, breaking, fixing, ruminating, succeeding, and having fun!

## Workflow


The creation of this app will take place in a single GitHub repo where you will *all* have push access (please don't abuse this, on the other hand, don't be afraid of making mistakes, we can always revert things!).  One big general git rule:

1. No force pushing on master. Ever. No exceptions.  If you need to get rid of a previous commit, then revert the commit by using `git revert`. 

The code review process will take place as follows:
1. Open an issue on the issue tracker, this is for both bugs and enhancements.  Assign yourself to this issue.  Dylan or Zach will assign another fellow to review your code.
2. Make a new branch (usually based off of master) and make a few commits.  Please do write good commit messages that actually mean something.
3. Open a Pull Request (usually against master) and request a review from both the other person assigned to the PR and either Dylan or Zach.  Assign both of these people to the PR.  
4. Your code gets reviewed by both the reviewers, politely have a conversation about changes if necessary.
5. When both reviewers are satisfied, and all tests pass, merge the pull request into master.  We prefer you use either the rebase and merge or squash and merge styles of merging.

The app will have CI set up to test and deploy the application to a Heroku server.

One big general workflow rule:

1. Be polite to your peers, give criticism respectfully and constructively.  Remember they have spent time writing that code, respect their time and effort.

## API


The model API is pretty simple, you give it a time series and a secret key and gives you back some predictions using [`fbprophet`](https://github.com/facebook/prophet).  The location of the running service as well as the secret key will be saved as environment variables on Heroku, *do not* put these into source control.  We don't want people who are not us to be able to hit our servers.  There are few reasons we are choosing to architect the application in this:

1. Its a good example of microservices.  Our main application shouldn't have to care exactly how these predictions are made, only the interface necessary to interact with the service that makes the predictions.
2. Heroku servers have pretty low RAM by default.  `fbprophet` (and many models) need more than 512 Mb.  Moving this to another server allows us to use much more RAM while still retaining the simplicity and ease of deploying to Heroku for our main application.
3. We want to protect the server address and secret key so that malicious users cannot overload our servers.

API spec:

This API exposes one endpoint ("/") and a post request of the following format for a training set of length N

```

{
  "key": Secret key
  "periods": Number of periods (usually days) to predict (default 100)
  "ds": [days] * N
  "y": [quantity to prediction] * N
}

```

and the return will be an object of the form

```
{
   "ds": {"index": Date}
   "yhat": {"index": predicted Value}
   "trend": {"index": Predicted Value}
   "trend_lower": {"index": Predicted Value}
   "trend_upper": {"index": Predicted Value}
   "yhat_lower": {"index": Predicted Value}
   "yhat_upper": {"index": Predicted Value}
   "seasonal": {"index": Predicted Value}
   "seasonal_lower": {"index": Predicted Value}
   "seasonal_upper": {"index": Predicted Value}
   "seasonalities": {"index": Predicted Value}
   "seasonalities_lower": {"index": Predicted Value}
   "seasonalities_upper": {"index": Predicted Value}
   "weekly": {"index": Predicted Value}
   "weekly_lower": {"index": Predicted Value}
   "weekly_upper": {"index": Predicted Value}
   "yearly": {"index": Predicted Value}
   "yearly_lower": {"index": Predicted Value}
   "yearly_upper": {"index": Predicted Value}
   "yhat": {"index": Predicted Value}
}

```

In order to understand this better better, we have made a mock-up of this API in `FO_TEST_API.ipynb`.  Run this notebook (after uncommenting the `app.run` command) and a local server will be started which exposes the same API.  This might be very useful as you are building your app, but don't want to hit the production modeling framework as that will generally take a bit longer to run.

In [ ]:
from datetime import datetime, timedelta
import random
import json
periods = 100
def gen_day(i):
    return str(datetime.now() + timedelta(days=i))

url = 'http://localhost:9000'

In [ ]:
import requests
def make_request(key, periods=100):
    data = {'ds':[gen_day(i) for i in range(100)],
            'y': [random.random() for _ in range(100)],
            'periods':periods}
    if key is not None:
        data['key'] = key
    try:
        r = requests.post(url, data=json.dumps(data))
    except requests.ConnectionError:
        return "make sure you have started a server on port 9000"
    if not r.ok:
        return "Error = {}, message = {}".format(r.status_code, r.text)
    return json.loads(r.text)

In [ ]:
make_request(None)

In [ ]:
make_request('random')

In [ ]:
make_request('I am a secret')

In [ ]:
make_request('I am a secret', 10)

## The big no-no


After reviewing your 12-Day projects, the biggest mistake we have noticed, is that people holding state within an application.  While this works just fine with only a single user and a single server (at least most of the time), it will break when running with multiple users or multiple servers.  For example, in `PR_Test_API.ipynb`, we have an example of this which we have suggestively called `bad_app`. 

First lets run a set of single queries and see what we get

In [ ]:
users = ['zach', 'dylan', 'robert']
for user in users:
    try:
        r = requests.get(url, params={'user': user})
    except requests.ConnectionError:
        print("make sure you have started a server on port 9000")
    else:
        print(user, r.text)

Things worked just fine here.  Now lets issue these requests, but we won't wait for the response before we issue the next one, this will simulate a multi-user environment.

In [ ]:
from requests_futures.sessions import FuturesSession
session = FuturesSession(max_workers=5)

futures = [session.get(url, params={'user': user}) for user in users]
result = []
for future in futures:
    try:
        result.extend(future.result())
    except requests.ConnectionError:
        print("make sure you have started a server on port 9000")
print(list(zip(users, result)))

*Copyright &copy; 2018 The Data Incubator.  All rights reserved.*